In [6]:
!pip install keras-tuner --upgrade

In [10]:
import tensorflow as tf
from tensorflow import keras
from keras import layers, Input
import keras_tuner as kt  # ✅ correct


In [9]:
import sys
print(sys.executable)


C:\Users\aajer\my_tensorflow_project\venv\Scripts\python.exe


In [11]:
# Load and preprocess dataset (MNIST)
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]
x_val, y_val = x_train[-10000:], y_train[-10000:]
x_train, y_train = x_train[:-10000], y_train[:-10000]


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step 


In [12]:
# =========================
# STEP 1: Search Space Setup
# =========================
search_space = {
    'filters': [32, 64, 96, 128],
    'kernel_size': [3, 5],
    'dense_units': [64, 128, 192, 256],
    'dropout': [0.2, 0.3, 0.4, 0.5],
    'learning_rate': [1e-2, 1e-3, 1e-4]
}

In [13]:

# ==========================================
# STEP 2: Wrap Training + Evaluation Function
# ==========================================
def build_model(hp):
    model = keras.Sequential()
    model.add(Input(shape=(28, 28, 1)))
    model.add(layers.Conv2D(
        filters=hp.Choice('filters', search_space['filters']),
        kernel_size=hp.Choice('kernel_size', search_space['kernel_size']),
        activation='relu'
    ))
    model.add(layers.MaxPooling2D())
    model.add(layers.Flatten())
    model.add(layers.Dense(
        units=hp.Choice('dense_units', search_space['dense_units']),
        activation='relu'
    ))
    model.add(layers.Dropout(
        hp.Choice('dropout', search_space['dropout'])
    ))
    model.add(layers.Dense(10, activation='softmax'))

    model.compile(
        optimizer=keras.optimizers.Adam(
            learning_rate=hp.Choice('learning_rate', search_space['learning_rate'])
        ),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

In [14]:
# ===================================
# STEP 3: Run the Hyperparameter Tuner
# ===================================
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    directory='my_dir',
    project_name='cnn_tuning_example'
)

In [15]:

# Start tuning
tuner.search(x_train, y_train, epochs=5, validation_data=(x_val, y_val))

Trial 5 Complete [00h 08m 38s]
val_accuracy: 0.989799976348877

Best val_accuracy So Far: 0.989799976348877
Total elapsed time: 00h 41m 18s


In [16]:

# Best model summary (optional)
best_model = tuner.get_best_models(num_models=1)[0]
best_model.summary()
# Show a summary of top trials
tuner.results_summary()

C:\Users\aajer\my_tensorflow_project\venv\lib\site-packages\keras\src\saving\saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 24, 24, 64)          │           1,664 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 12, 12, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 9216)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │       1,179,776 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 10)                  │           1,290 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,182,730 (4.51 MB)

 Trainable params: 1,182,730 (4.51 MB)

 Non-trainable params: 0 (0.00 B)

Results summary
Results in my_dir\cnn_tuning_example
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 4 summary
Hyperparameters:
filters: 64
kernel_size: 5
dense_units: 128
dropout: 0.2
learning_rate: 0.001
Score: 0.989799976348877

Trial 3 summary
Hyperparameters:
filters: 64
kernel_size: 3
dense_units: 128
dropout: 0.3
learning_rate: 0.001
Score: 0.9873999953269958

Trial 0 summary
Hyperparameters:
filters: 64
kernel_size: 3
dense_units: 192
dropout: 0.3
learning_rate: 0.01
Score: 0.9818999767303467

Trial 2 summary
Hyperparameters:
filters: 128
kernel_size: 5
dense_units: 192
dropout: 0.5
learning_rate: 0.01
Score: 0.9797999858856201

Trial 1 summary
Hyperparameters:
filters: 32
kernel_size: 3
dense_units: 128
dropout: 0.5
learning_rate: 0.01
Score: 0.9793999791145325


In [17]:
# Print accuracy of all trials
print("\nAll Trial Accuracies:")
for trial in tuner.oracle.get_best_trials(num_trials=len(tuner.oracle.trials)):
    val_acc = trial.metrics.get_last_value('val_accuracy')
    print(f"Trial ID: {trial.trial_id} | Validation Accuracy: {val_acc:.4f}")


All Trial Accuracies:
Trial ID: 4 | Validation Accuracy: 0.9898
Trial ID: 3 | Validation Accuracy: 0.9874
Trial ID: 0 | Validation Accuracy: 0.9819
Trial ID: 2 | Validation Accuracy: 0.9798
Trial ID: 1 | Validation Accuracy: 0.9794
